## Quadratic Logistic Model, d-optimal

In [1]:
using PyPlot
using LinearAlgebra

# projection function onto the design space
function proj(x, design)
    n = Int((length(x)+1)/2)
    a = x[1:n]
    b = x[(n+1):(2*n-1)]
    b = [b; 1-sum(b)]
    
    a = max.(a,design[1])
    a = min.(a,design[2])
    
    u = sort(b, rev=true)
    j = n
    while true
        if u[j] + 1/j*(1-cumsum(u)[j]) > 0
            break
        end
        j -= 1
    end
    lambda = 1/j*(1-cumsum(u)[j])
    b = max.(b .+ lambda,0)
    b = b[1:(n-1)]
    
    return [a;b]
end

proj (generic function with 1 method)

In [2]:
#Information matrix for quadratic logistic model
function infmat(x)
    n = Int((length(x)+1)/2)
    a = x[1:n]
    b = x[(n+1):(2*n-1)]
    b = [b; 1-sum(b)]
    p = length(theta)
    mat = zeros(p,p)
    for i in 1:n
        c = exp(theta[1]+theta[2]*(a[i]-theta[3])^2)
        f = [1,(a[i]-theta[3])^2,2*theta[2]*(theta[3]-a[i]) ]
        mat = mat + b[i] * c/(1+c)^2 * f*f'
    end
    return mat
end

infmat (generic function with 1 method)

In [3]:
function diter(k, nptc, niter, design)
    #initialize
    #srand(1992)
    tau = 0.9:(-0.5/niter):0.4
    v = zeros(nptc, 2k-1)
    x = (design[2]-design[1])*rand(nptc*k) .+ design[1]
    x = reshape(x,nptc,k)
    w = rand(nptc*(k-1))
    w = reshape(w,nptc,k-1)
    ptc = hcat(x,w)
    for j in 1:nptc
        ptc[j,:] = proj(ptc[j,:],design)
    end
    pbest = ptc
    pbesth = zeros(nptc)
    for i in 1:nptc
        pbesth[i] = det(infmat(pbest[i,:]))
    end
    gbesth = findmax(pbesth)[1]
    gbest = pbest[findmax(pbesth)[2],:]

    #iterate
    for i in 1:niter # iteration number
        for j in 1:nptc # particle number
            v[j,:] = tau[i] .*v[j,:] + 2 .*rand(2k-1) .*(pbest[j,:]-ptc[j,:]) + 2 .*rand(2k-1) .*(gbest-ptc[j,:])
            ptc[j,:] = ptc[j,:] + v[j,:]
            ptc[j,:] = proj(ptc[j,:],design)
            fit = det(infmat(ptc[j,:]))
            if fit > gbesth
                gbest = pbest[j,:] = ptc[j,:]
                gbesth = pbesth[j] = fit
            elseif fit > pbesth[j]
                pbest[j,:] = ptc[j,:]
                pbesth[j] = fit
            end
        end
    end
    supp = gbest[1:k]
    prob = gbest[(k+1):2k-1]
    prob = [prob; 1-sum(prob)]
    return [supp, prob, gbesth]
end

diter (generic function with 1 method)

In [4]:
theta = [2,3,0]

diter(3, 128, 150, [-3,1])

3-element Array{Any,1}:
  [-0.726988, 1.54538e-9, 0.726988]
  [0.333333, 0.333333, 0.333333]   
 5.694646494958843e-5              

In [5]:
nptc = [64, 128, 256, 512, 1024]
niter = [50, 100, 200, 500, 1000]
nsim = 200

result = zeros(5,5)
result2 = zeros(5,5)
timeresult = zeros(5,5)
for i in 1:5
    for j in 1:5
        temp = 0
        temp2 = 0
        t1 = time()
        for k in 1:nsim
            opt = diter(3, nptc[i], niter[j], [-3,1])[3]
            if opt > 0.99 * 5.69464649495885e-5
                temp += 1
            end
            if opt > 0.995 * 5.69464649495885e-5
                temp2 += 1
            end
        end
        t2 = time()
        result[i,j] = temp/nsim
        result2[i,j] = temp2/nsim
        timeresult[i,j] = (t2-t1)/nsim
        println("simulation saved")
    end
end

simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved


In [6]:
display(result)
display(timeresult)

5×5 Array{Float64,2}:
 0.45   0.485  0.47   0.52   0.49 
 0.735  0.675  0.72   0.74   0.675
 0.895  0.915  0.91   0.895  0.94 
 0.995  0.975  0.995  0.99   0.99 
 1.0    1.0    1.0    1.0    1.0  

5×5 Array{Float64,2}:
 0.064083  0.125632  0.250002  0.623356   1.24469
 0.126409  0.250797  0.500852  1.2505     2.49776
 0.247339  0.4815    0.958213  2.3749     4.74953
 0.486653  0.96712   1.91749   4.75633    9.51685
 0.973217  1.92149   3.82201   9.5445    19.4257 

In [7]:
theta = [3,-5,0]

result = diter(4, 256, 200, [-1,1])

3-element Array{Any,1}:
  [-0.826012, -0.57935, 0.848815, -1.0]      
  [2.97374e-16, 0.333333, 0.333333, 0.333333]
 0.018228167200968263                        

In [8]:
result = zeros(5,5)
timeresult = zeros(5,5)
for i in 1:4
    for j in 1:4
        temp = 0
        t1 = time()
        for k in 1:nsim
            opt = diter(4, nptc[i], niter[j], [-1,1])[3]
            if opt > 0.99 * 0.02023431778515896
                temp += 1
            end
        end
        t2 = time()
        result[i,j] = temp/nsim
        timeresult[i,j] = (t2-t1)/nsim
        println("simulation saved")
    end
end

simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved
simulation saved


In [9]:
display(result)
display(timeresult)     

5×5 Array{Float64,2}:
 0.035  0.035  0.04   0.045  0.0
 0.055  0.055  0.04   0.035  0.0
 0.07   0.06   0.055  0.06   0.0
 0.08   0.135  0.115  0.09   0.0
 0.0    0.0    0.0    0.0    0.0

5×5 Array{Float64,2}:
 0.0724478  0.142262  0.284356  0.706746  0.0
 0.144571   0.286477  0.573935  1.4284    0.0
 0.288331   0.574176  1.1556    3.04962   0.0
 0.622514   1.24386   2.44147   6.08171   0.0
 0.0        0.0       0.0       0.0       0.0